In [1]:
from typing import Union, Self
from numbers import Number
from itertools import chain, product
import pickle
from sparseqr import qr

import numpy as np
import sympy as sp
import cvxpy as cp
import json
import hashlib
import scipy
from scipy.sparse import csr_matrix
import scipy.sparse as sparse
from scipy.sparse import coo_matrix, csc_matrix
from scipy.sparse.linalg import splu, svds
from sksparse.cholmod import cholesky
import xxhash
from bmn.algebra import MatrixOperator, SingleTraceOperator, MatrixSystem, DoubleTraceOperator
from bmn.linear_algebra import get_null_space_dense, create_sparse_matrix_from_dict, get_row_space_dense, get_null_space_sparse
from bmn.bootstrap import BootstrapSystem
from bmn.solver_trustregion import (
    minimal_eigval,
    sdp_init, sdp_relax,
    sdp_minimize,
    #minimize,
    get_quadratic_constraint_vector_sparse,
    get_quadratic_constraint_vector_dense,
    compute_L2_norm_of_quadratic_constraints,
)
from bmn.solver_pytorch import solve_bootstrap as solve_bootstrap_pytorch
from bmn.models import OneMatrix, TwoMatrix, MiniBFSS
from bmn.brezin import compute_Brezin_energy, compute_Brezin_energy_Han_conventions
import os
from bmn.debug_utils import disable_debug
from bmn.solver_newton import solve_bootstrap_Ax_eq_b

import yaml
from bmn.config_utils import (
    run_bootstrap_from_config,
    generate_configs_one_matrix,
    generate_configs_two_matrix,
    generate_configs_three_matrix,
    generate_configs_bfss,
    )

from bmn.models import OneMatrix, TwoMatrix, MiniBFSS
import pandas as pd
from bmn.debug_utils import debug


# plot settings
import matplotlib.pyplot as plt
import matplotlib
from cycler import cycler
import torch
import torch.optim as optim
from torch.nn import ReLU

plt.rcParams['xtick.direction'] = 'in'
plt.rcParams['ytick.direction'] = 'in'
plt.rcParams['xtick.major.size'] = 5.0
plt.rcParams['xtick.minor.size'] = 3.0
plt.rcParams['ytick.major.size'] = 5.0
plt.rcParams['ytick.minor.size'] = 3.0
plt.rcParams['lines.linewidth'] = 2
plt.rc('font', family='serif',size=16)
matplotlib.rc('text', usetex=True)
matplotlib.rc('legend', fontsize=16)
matplotlib.rcParams['axes.prop_cycle'] = cycler(
    color=['#E24A33', '#348ABD', '#988ED5', '#777777', '#FBC15E', '#8EBA42', '#FFB5B8']
    )
matplotlib.rcParams.update(
    {"axes.grid":False,
    "grid.alpha":0.75,
    "grid.linewidth":0.5}
    )
colors = plt.rcParams['axes.prop_cycle'].by_key()['color']

np.set_printoptions(linewidth=120)  # Adjust the number to the desired width
torch.set_printoptions(linewidth=120)  # Adjust the number to the desired width

## [dev] - new cvxpy optmization

In [2]:
L = 3
g2 = 0

model = TwoMatrix(couplings={"g2": 0, "g4": 1})
checkpoint_path = f"checkpoints/TwoMatrix_L_{L}_symmetric_energy_fixed_g2_{g2}"

#model = MiniBFSS(couplings={"lambda": 1})
#checkpoint_path = f"checkpoints/MiniBFSS_L_{L}_symmetric"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=L,
    symmetry_generators=model.symmetry_generators,
    checkpoint_path=checkpoint_path,
    verbose=False,
    )
bootstrap.load_constraints(path=bootstrap.checkpoint_path)

Assuming all operators are either Hermitian or anti-Hermitian.
NOTE Remember to incorporate more general basis changes!
Bootstrap system instantiated for 5461 operators
Attribute: simplify_quadratic = True
Attempting to load from checkpoints, checkpoint_path=checkpoints/TwoMatrix_L_3_symmetric_energy_fixed_g2_0
  loaded previously computed linear constraints
  loaded previously computed cyclic constraints
  loaded previously computed null space matrix
  loaded previously computed quadratic constraints (numerical)
  loaded previously computed bootstrap table


In [6]:
energy = 2.4

param, optimization_result = solve_bootstrap_Ax_eq_b(
    bootstrap=bootstrap,
    st_operator_to_minimize=model.operators_to_track["x_2"],
    init=None,
    init_scale=1e-4,
    st_operator_inhomo_constraints=[
        (SingleTraceOperator(data={(): 1}), 1),
        (model.operators_to_track["energy"], energy),],
    radius=5e10,
    #maxiters_cvxpy=2_000_000,
    maxiters_cvxpy=100_000,
    reg=1e8,
    eps_abs=1e-9,
    eps_rel=1e-9,
    eps_infeas=1e-9,
    tol=1e-8,
    )

DEBUG 18:44:49.590013: Using method newton_Axb
DEBUG 18:44:49.590052: Final bootstrap parameter dimension: 71
DEBUG 18:44:49.590688: 

step: 1/25
DEBUG 18:44:49.590709: PRNG seed: None
DEBUG 18:44:49.590723: radius: 5.0000e+10
DEBUG 18:44:49.590733: reg: 1.0000e+08
DEBUG 18:44:49.590742: eps_abs: 1.0000e-09
DEBUG 18:44:49.590750: eps_rel: 1.0000e-09
DEBUG 18:44:49.590758: eps_infeas: 1.0000e-09
DEBUG 18:44:49.590768: tol: 1.0000e-08
DEBUG 18:44:49.590776: cvxpy_solver: SCS
DEBUG 18:44:49.590790: st_operator_inhomo_constraints: 1 <tr()>, val=1.0000
DEBUG 18:44:49.590812: st_operator_inhomo_constraints: -0.5 <tr(Pi0Pi0)> -0.5 <tr(Pi1Pi1)> -0.25 <tr(X0X1X0X1)> -0.25 <tr(X1X0X1X0)> + 0.25 <tr(X0X1X1X0)> + 0.25 <tr(X1X0X0X1)>, val=2.4000
DEBUG 18:44:49.590824: st_op_to_minimize: 1 <tr(X0X0)> + 1 <tr(X1X1)>
DEBUG 18:45:58.271950: sdp_minimize status after maxiters_cvxpy 100000: optimal_inaccurate
DEBUG 18:45:58.272026: sdp_minimize ||A x - b||: 4.4409e-16
DEBUG 18:45:58.272035: sdp_minimize 

SolverError: Solver 'SCS' failed. Try another solver, or solve with verbose=True for more information.

In [ ]:
st_op = SingleTraceOperator(data={("Pi0",): -1j, ("X0",): -1j, ("X1",): -1, ("Pi1",): -1})

bootstrap.matrix_system.single_trace_commutator(
    model.symmetry_generators[0],
    st_op,
    ) + 1j*st_op

In [ ]:
bootstrap.get_operator_expectation_value(
    model.operators_to_track["x_2"],
    param,
)

In [ ]:
bootstrap.get_operator_expectation_value(
    model.operators_to_track["energy"],
    param,
)

## explore Hessian

In [ ]:
L = 3

model = MiniBFSS(couplings={"lambda": 1})
checkpoint_path = f"checkpoints/MiniBFSS_L_{L}_symmetric"

#model = TwoMatrix(couplings={"g2": 0, "g4": 1})
#checkpoint_path = f"checkpoints/TwoMatrix_L_{L}_energy_fixed_g2_0.1"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=L,
    symmetry_generators=model.symmetry_generators,
    checkpoint_path=checkpoint_path,
    verbose=False,
    )
bootstrap.load_constraints(path=bootstrap.checkpoint_path)

In [ ]:
energy = 1.866667
#energy = 1.333333
#energy = 1.533333

penalty_reg = 0
st_operator_to_minimize = model.operators_to_track["x_2"]
st_operator_inhomo_constraints = [
    (SingleTraceOperator(data={(): 1}), 1),
    (model.operators_to_track["energy"], energy),
    ]

In [ ]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
PRNG_seed = None
print(f"torch device: {device}")


if PRNG_seed is not None:
    np.random.seed(PRNG_seed)
    torch.manual_seed(PRNG_seed)
    debug(f"setting PRNG seed to {PRNG_seed}")

# get the bootstrap constraints necessary for the optimization
# linear constraints
if bootstrap.linear_constraints is None:
    _ = bootstrap.build_linear_constraints().tocsr()

# quadratic constraints
if bootstrap.quadratic_constraints_numerical is None:
    bootstrap.build_quadratic_constraints()

# bootstrap table
if bootstrap.bootstrap_table_sparse is None:
    bootstrap.build_bootstrap_table()
debug(f"Final bootstrap parameter dimension: {bootstrap.param_dim_null}")

# build the Ax = b constraints
A, b = [], []
for st_operator, val in st_operator_inhomo_constraints:
    A.append(
        bootstrap.single_trace_to_coefficient_vector(
            st_operator, return_null_basis=True
        )
    )
    b.append(val)
A = np.asarray(A)  # convert to numpy array
b = np.asarray(b)

A_null_space = get_null_space_dense(matrix=A)
null_space_projector = 0*np.eye(A.shape[1]) + A_null_space @ np.linalg.pinv(A_null_space)

A = torch.from_numpy(A).type(torch.float).to(device)  # convert to torch tensor
b = torch.from_numpy(b).type(torch.float).to(device)
null_space_projector = torch.from_numpy(null_space_projector).type(torch.float).to(device)

#eturn null_space_projector

# get the vector of the operator to bound (minimize)
vec = bootstrap.single_trace_to_coefficient_vector(
    st_operator_to_minimize, return_null_basis=True
)
vec = torch.from_numpy(vec).type(torch.float).to(device)

# build the bootstrap array
bootstrap_array_torch = (
    torch.from_numpy(bootstrap.bootstrap_table_sparse.todense())
    .type(torch.float)
    .to(device)
)

# build the constraints
quadratic_constraints = bootstrap.quadratic_constraints_numerical
quadratic_constraint_linear = (
    torch.from_numpy(quadratic_constraints["linear"].todense())
    .type(torch.float)
    .to(device)
)
quadratic_constraint_quadratic = (
    torch.from_numpy(quadratic_constraints["quadratic"].todense())
    .type(torch.float)
    .to(device)
)
quadratic_constraint_quadratic = quadratic_constraint_quadratic.reshape(
    (
        len(quadratic_constraint_quadratic),
        bootstrap.param_dim_null,
        bootstrap.param_dim_null,
    )
)

def operator_loss(param_null, param_particular):
    param = null_space_projector @ param_null + param_particular
    return vec @ param

def get_quadratic_constraint_vector(param):
    quadratic_constraints = torch.einsum(
        "Iab, a, b -> I", quadratic_constraint_quadratic, param, param
    ) + torch.einsum("Ia, a -> I", quadratic_constraint_linear, param)
    return torch.square(quadratic_constraints)

def quadratic_loss(param_null, param_particular):
    param = null_space_projector @ param_null + param_particular
    return torch.norm(get_quadratic_constraint_vector(param))

def Axb_loss(param_null, param_particular):
    param = null_space_projector @ param_null + param_particular
    return torch.norm(A @ param - b)

def psd_loss(param_null, param_particular):
    param = null_space_projector @ param_null + param_particular
    bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )
    smallest_eigv = torch.linalg.eigvalsh(bootstrap_matrix)[0]
    return ReLU()(-smallest_eigv)

def build_loss(param_null, param_particular, penalty_reg=penalty_reg):
    loss = (
        operator_loss(param_null, param_particular)
        + penalty_reg * psd_loss(param_null, param_particular)
        + penalty_reg * quadratic_loss(param_null, param_particular)
        + penalty_reg * Axb_loss(param_null, param_particular)
    )
    return loss

def num_zero_eigenvalues(param_null, param_particular, tol=1e-5):
    param = null_space_projector @ param_null + param_particular
    bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )
    return torch.sum(torch.abs(torch.linalg.eigvalsh(bootstrap_matrix)) < tol).cpu().detach().item()

def bootstrap_eigenvalues(param_null, param_particular, tol=1e-5):
    param = null_space_projector @ param_null + param_particular
    bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )
    return torch.linalg.eigvalsh(bootstrap_matrix).cpu().detach().numpy()

In [ ]:
datadir = f"data/MiniBFSSx_L_3_symmetric_energy_fixed"
datadir = f"data/MiniBFSSx_L_3_symmetric_energy_fixed_pytorch"
file = f"energy_{energy}_op_to_min_x_2.json"

with open(f"{datadir}/{file}") as f:
    result = json.load(f)

In [ ]:
param = torch.tensor(result["param"]).type(torch.float).to(device)
param_particular = torch.tensor(np.linalg.lstsq(A.cpu().numpy(), b.cpu().numpy())[0]).type(torch.float).to(device)
param_null = param - param_particular

In [ ]:
bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )
torch.set_printoptions(profile="full")


In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
ax.grid(False)

for idx, method in enumerate(['cvxpy', 'pytorch']):

    # load
    datadir = f"data/MiniBFSSx_L_3_symmetric_energy_fixed"
    if method == 'pytorch':
        datadir += "_pytorch"
    file = f"energy_{energy}_op_to_min_x_2.json"
    with open(f"{datadir}/{file}") as f:
        result = json.load(f)

    # build param
    param = torch.tensor(result["param"]).type(torch.float).to(device)
    param_particular = torch.tensor(np.linalg.lstsq(A.cpu().numpy(), b.cpu().numpy())[0]).type(torch.float).to(device)
    param_null = param - param_particular

    # plot CDF
    data = bootstrap_eigenvalues(param_null, param_particular)
    x, CDF_counts = np.unique(data, return_counts = True)
    y = np.cumsum(CDF_counts)/np.sum(CDF_counts)
    ax.plot(x, y, label=method)

    fraction_near_zero = num_zero_eigenvalues(param_null, param_particular, tol=1e-2) / bootstrap.bootstrap_matrix_dim
    ax.axhline(fraction_near_zero, color=colors[idx], linewidth=1, linestyle='--')
    print(f"fraction of almost zero eigs: {fraction_near_zero:.4f}")

ax.set_title(f"energy = {energy}")
ax.set_xscale('log')
ax.legend()
plt.show()

In [ ]:
# add a random null param vector for comparison
param_null = 1e4 * torch.randn_like(param_null)
data = bootstrap_eigenvalues(param_null, param_particular)
print(f"number of near-zero eigenvalues: {np.sum(np.abs(data) < 1e-6)}")

In [ ]:
bootstrap.bootstrap_matrix_dim

In [ ]:
fig, ax = plt.subplots(1, 3, figsize=(12, 6))
for idx, method in enumerate(['cvxpy', 'pytorch', 'random']):

    # load
    if method == 'random':
        param = torch.randn(bootstrap.param_dim_null).to(device)
    else:
        datadir = f"data/MiniBFSSx_L_3_symmetric_energy_fixed"
        if method == 'pytorch':
            datadir += "_pytorch"
        file = f"energy_{energy}_op_to_min_x_2.json"
        with open(f"{datadir}/{file}") as f:
            result = json.load(f)
        # build param
        param = torch.tensor(result["param"]).type(torch.float).to(device)

    bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )
    mat = bootstrap_matrix.cpu().detach().numpy()

    #print(np.sum(mat))
    mat = np.abs(mat) / np.max(mat)
    mat = 1e-10 + mat
    mat = np.log(mat)
    #mat = 1 - mat
    #mat = np.log(np.abs(mat))
    #mat = np.arctanh(mat)

    if method == "cvxpy":
        mat_cvxpy = mat
    elif method == 'pytorch':
        mat_pytorch = mat
    else:
        mat_random = mat

    ax[idx].imshow(mat, cmap='binary')
    ax[idx].set_title(f"{method}")

plt.savefig(f"figures/matrix.png", dpi=600)
plt.show()

In [ ]:
bootstrap.bootstrap_matrix_dim**2

The matrix dim is 259

| degree      | number |
| ----------- | ----------- |
| 0      | 1       |
| 1   | 6        |
| 2   | 36        |
| 3   | 216        |

1 + 6 + 36 + 216 = 259

even operators can be made by even * even (37 options) or odd * odd (217 options)

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
#mat_diff = np.abs(mat_cvxpy)
ax.imshow(mat_cvxpy[0:37, 0:37], cmap='binary')
plt.savefig(f"figures/matrix_difference.png", dpi=600)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
#mat_diff = np.abs(mat_cvxpy)
ax.imshow(mat_cvxpy[38:, 38:], cmap='binary')
plt.savefig(f"figures/matrix_difference.png", dpi=600)
plt.show()

In [ ]:
tol = 1e-4
method = 'cvxpy'
param_list = []
energies = [1.333333, 1.533333, 1.866667]
for energy in energies:

    st_operator_to_minimize = model.operators_to_track["x_2"]
    st_operator_inhomo_constraints = [
        (SingleTraceOperator(data={(): 1}), 1),
        (model.operators_to_track["energy"], energy),
        ]

    # build the Ax = b constraints
    A, b = [], []
    for st_operator, val in st_operator_inhomo_constraints:
        A.append(
            bootstrap.single_trace_to_coefficient_vector(
                st_operator, return_null_basis=True
            )
        )
        b.append(val)
    A = np.asarray(A)  # convert to numpy array
    b = np.asarray(b)

    A_null_space = get_null_space_dense(matrix=A)
    null_space_projector = 0*np.eye(A.shape[1]) + A_null_space @ np.linalg.pinv(A_null_space)

    A = torch.from_numpy(A).type(torch.float).to(device)  # convert to torch tensor
    b = torch.from_numpy(b).type(torch.float).to(device)
    null_space_projector = torch.from_numpy(null_space_projector).type(torch.float).to(device)

    # build the bootstrap array
    bootstrap_array_torch = (
        torch.from_numpy(bootstrap.bootstrap_table_sparse.todense())
        .type(torch.float)
        .to(device)
    )

    # load
    datadir = f"data/MiniBFSSx_L_3_symmetric_energy_fixed"
    if method == 'pytorch':
        datadir += "_pytorch"
    file = f"energy_{energy}_op_to_min_x_2.json"
    with open(f"{datadir}/{file}") as f:
        result = json.load(f)

    # build param
    param = torch.tensor(result["param"]).type(torch.float).to(device)
    param_particular = torch.tensor(np.linalg.lstsq(A.cpu().numpy(), b.cpu().numpy())[0]).type(torch.float).to(device)
    param_null = param - param_particular

    param_list.append(param.cpu().detach().numpy())

    # build the bootstrap matrix
    bootstrap_matrix = (bootstrap_array_torch @ param).reshape(
        (bootstrap.bootstrap_matrix_dim, bootstrap.bootstrap_matrix_dim)
    )
    bootstrap_matrix_even = bootstrap_matrix[0:37, 0:37]
    bootstrap_matrix_odd = bootstrap_matrix[37:, 37:]

    num_zero_eigs = torch.sum(torch.abs(torch.linalg.eigvalsh(bootstrap_matrix)) < tol).cpu().detach().item()
    num_zero_eigs_even = torch.sum(torch.abs(torch.linalg.eigvalsh(bootstrap_matrix_even)) < tol).cpu().detach().item()
    num_zero_eigs_odd = torch.sum(torch.abs(torch.linalg.eigvalsh(bootstrap_matrix_odd)) < tol).cpu().detach().item()

    print(f"num_zero_eigs: {num_zero_eigs}")
    print(num_zero_eigenvalues(param_null, param_particular, tol=1e-4)/259)
    #print(f"num_zero_eigs_even: {num_zero_eigs_even}")
    #print(f"num_zero_eigs_odd: {num_zero_eigs_odd}")

param_list = np.asarray(param_list)

In [ ]:
0.2046

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))

n = 5
for i in range(n):
    ax.plot(energies, param_list[:,i], '-o')
plt.show()

In [ ]:
op = ('X0', 'X0', 'Pi0', 'X0')
idx = bootstrap.operator_dict[op]

for op in bootstrap.operator_list:
    if len(op) == 6:
        dim_appearances = ["".join(op).count(str(i)) for i in range(3)]
        if all([d % 2 == 0 for d in dim_appearances]):

            #if "".join(op).count('X0') == 6:
            if op == ("X0", "X0", "X1", "X1", "X2", "X2"):
                lhs, rhs = bootstrap.generate_cyclic_constraint(op)

                # set odd degree terms to zero
                #lhs_new = SingleTraceOperator(data={key: value for key, value in lhs if len(key) % 2 == 0})
                lhs_new = lhs
                rhs_new = DoubleTraceOperator(data={key: value for key, value in rhs if (len(key[0]) % 2 == 0) and (len(key[1]) % 2 == 0)})
                #degrees = [(len(term1), len(term2)) for (term1, term2) in rhs.data.keys()]

                #if any([(deg1 % 2 == 0) and (deg2 % 2 == 0) for (deg1, deg2) in degrees]):
                print(f"op: {op}")
                print(f"LHS: {lhs_new}")
                print(f"RHS: {rhs_new}")
                print()

In [ ]:
bootstrap.generate_cyclic_constraint(('Pi0', 'X0', 'X0', 'X0', 'X0', 'X0'))

In [ ]:
bootstrap.generate_cyclic_constraint(('X0', 'Pi0', 'X0', 'X0', 'X0', 'X0'))

In [ ]:
bootstrap.generate_cyclic_constraint(('X0', 'X0', 'X0', 'Pi0', 'X0', 'X0'))

In [ ]:
data_dir = "data/MiniBFSSx_L_3_symmetric_energy_fixed/energy_1.2_op_to_min_x_2.json"
with open(data_dir) as f:
    result = json.load(f)
param = np.asarray(result['param'])

In [ ]:
x4 = bootstrap.get_operator_expectation_value(
    st_operator=SingleTraceOperator(data={("X0", "X0", "X0", "X0"): 1}),
    param=param,
)

xy_commutator = bootstrap.get_operator_expectation_value(
    st_operator=SingleTraceOperator(
        data={
            ("X0", "X1", "X0", "X1"): 1,
            ("X1", "X0", "X1", "X0"): 1,
            ("X0", "X1", "X1", "X0"): 1,
            ("X1", "X0", "X0", "X1"): 1,
            }
        ),
    param=param,
)

In [ ]:
x4, xy_commutator

In [ ]:
L = 4
model = MiniBFSS(couplings={"lambda": 1})
checkpoint_path = f"checkpoints/MiniBFSS_L_{L}_lazy_symmetric"

bootstrap = BootstrapSystem(
    matrix_system=model.matrix_system,
    hamiltonian=model.hamiltonian,
    gauge_generator=model.gauge_generator,
    max_degree_L=L,
    symmetry_generators=model.symmetry_generators,
    verbose=True,
    checkpoint_path=checkpoint_path,
    odd_degree_vanish=True,
    #load_from_previously_computed=True
    )
bootstrap.load_constraints(checkpoint_path)

In [ ]:
bootstrap.cyclic_quadratic

In [ ]:
import json
json.dumps({ "x": 12153535.232321, "y": 35234531.232322 })

In [ ]:
x3 = 0.5664224028587341
x4 = 0.5642986304623219
x3 - x4

In [ ]:
len(bootstrap.linear_constraints)

In [ ]:
new_constraints = []

set_of_hashes = set()
for st_operator in bootstrap.linear_constraints:
    new_dict = {}
    norm = 1
    for idx, (k, v) in enumerate(st_operator):
        if idx == 0:
            norm = float(1/v)
        new_dict[''.join(k)] = float(v) * norm
    #new_dict = {''.join(k): float(v) for k, v in st_operator}
    s = json.dumps(new_dict).encode('utf-8')
    hash_digest = hashlib.md5(s).digest()
    if hash_digest not in set_of_hashes:
        new_constraints.append(st_operator)
        set_of_hashes.add(hash_digest)

In [ ]:
print(f"number of unique hashes: {len(set_of_hashes)}, ratio of hashes to constraints: {len(set_of_hashes) / len(bootstrap.linear_constraints)}")

In [ ]:
bootstrap.linear_constraints = new_constraints

In [ ]:
linear_constraint_matrix = bootstrap.build_linear_constraints()

In [ ]:
linear_constraint_matrix.shape

In [ ]:
bootstrap.build_null_space_matrix()